In [1]:
import afqinsight as afqi
import numpy as np
import pandas as pd

from collections import defaultdict
from keras_ssg_lasso import sgl
from tqdm import tqdm

from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [1]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1395969844049628241
]


In [ ]:
nodes = pd.read_csv('../AFQ-Insight/afqinsight/data/nodes.csv')
targets = pd.read_csv('../AFQ-Insight/afqinsight/data/subjects.csv', index_col='subjectID').drop(['Unnamed: 0'], axis='columns')
y = targets['class']
y = y.map(lambda h: int(h == 'ALS')).values

In [ ]:
transformer = afqi.AFQFeatureTransformer()
x, groups, cols = transformer.transform(nodes)

In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.15, stratify=y
)

In [ ]:
ind_sparse = np.ones_like(groups)
d = x.shape[1]

In [ ]:
y_train

In [ ]:
def create_classification_model(alpha=0.1, lambda_=0.1, n_epochs=100):
    model = sgl.SSGL_LogisticRegression(
        dim_input=d, n_classes=2, groups=groups, indices_sparse=ind_sparse,
        alpha=alpha, lambda_=lambda_, n_epochs=n_epochs, optimizer='adam', lr=0.001,
        validation_split=0.0, early_stopping_patience=0,
        # verbose=True
    )    
    return model.model

model = KerasClassifier(
    build_fn=create_classification_model,
    verbose=0
)

In [ ]:
# monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, verbose=1, mode='auto')
# checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True) # save best model

# model.fit(x_train, y_train, validation_data=(x_test, y_test),
#           callbacks=[monitor, checkpointer],
#           verbose=1, epochs=1000, batch_size=16)

In [ ]:
# define the grid search parameters
# batch_size = [32, 64]
epochs = [5, 10, 20, 50]
alphas = np.array([0.05, 0.5, 0.95])
lambdas = np.logspace(-4, 4, 20)
param_grid = dict(n_epochs=epochs, alpha=alphas, lambda_=lambdas)
cv_generator = RepeatedStratifiedKFold(n_splits=3, n_repeats=3)
scoring = {'AUC': 'roc_auc', 'Accuracy': 'accuracy'}
grid = GridSearchCV(
    estimator=model, param_grid=param_grid,
    scoring=scoring, refit='AUC', 
    cv=cv_generator, verbose=5
)

In [ ]:
grid_result = grid.fit(x_train, y_train)

In [ ]:
best_model = grid.best_estimator_.model
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True) # save best model

best_model.fit(x,y,validation_data=(x_test,y_test),callbacks=[monitor, checkpointer],verbose=1,epochs=1000)
best_model.load_weights('best_weights.hdf5') # load weights from best model
pred = best_model.predict(x_test)
pred_all = best_model.predict(x)

In [ ]:
pred_all > 0.5

In [ ]:
beta_hat = best_model.get_weights()[0]

In [ ]:
print('##.   coef  important  > 10^-3')
print('--- ------- --------- ---------')
for i, (b, beta) in enumerate(zip(idx, beta_hat[:, 0])):
    print('{:02d}. {:+6.4f}   {:5s}     {:5s}'.format(i, beta, str(b), str(abs(beta) >= 1e-2)))

In [ ]:
y_pred = pred > 0.5

In [ ]:
for yt, yp in zip(y_test, y_pred):
    print(yt, yp)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
best_model = grid.best_estimator_.model

In [ ]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True) # save best model

best_model.fit(x,y_class,validation_data=(x_test,y_test),callbacks=[monitor, checkpointer],verbose=1,epochs=1000)
best_model.load_weights('best_weights.hdf5') # load weights from best model
pred = best_model.predict(x_test)

In [ ]:
print(pred)
print(y_test)